In [1]:

import nltk, string
from sklearn.feature_extraction.text import TfidfVectorizer

import funclib.pandaslib as pandaslib
import xlwings

import pandas as pd
import funclib.baselib as baselib
import funclib.iolib as iolib


class Game():
    '''game cls'''
    def __init__(self, url, full_title, score=0, votes=0, year_released=None):
        if not isinstance(full_title, str): raise ValueError('Full title was not a string')
        if full_title is None: raise ValueError('Full title was None')
        

        self.full_title = full_title.lower()
        self.year_released = year_released
        self.score = score
        self.votes = votes
        self.url = url
  
        
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)

def normalize(text):
    stem_t = text.lower().translate(remove_punctuation_map)
    return stem_t

vectorizer = TfidfVectorizer(tokenizer=normalize)

def cosine_sim(text1, text2):
    tfidf = vectorizer.fit_transform([text1, text2])
    return ((tfidf * tfidf.T).A)[0,1]

In [2]:
games_root = 'C:/temp/c64/Games'
copy_games_to = 'C:/temp/c64_best/Games'

games = baselib.unpickle('c:/temp/c64games.pkl')
games_names = []
games_scores = []
games_votes = []
games_val = []
games_yr = []

exclude = ['german']



for g in games:
    skip = False
    if ((g.score > 7.5 and (g.score * g.votes > 200)) or (g.score * g.votes > 600)) and g.year_released < 1992:
        for s in exclude:
            if s in g.full_title:
                skip = True
                break
        if skip: continue
        games_names.append(g.full_title)
        games_scores.append(g.score)
        games_votes.append(g.votes)
        games_val.append(g.votes * g.score)
        games_yr.append(g.year_released)
    

                            
game_folders = []
game_folders_full = []

for folder, filename, extension, fullname in iolib.file_list_generator_dfe(games_root, '*.*', recurse=True):
    fld = folder.split('\\')[-1]
    if len(fld) == 1: continue
    game_folders.append(fld)
    game_folders_full.append(folder)
    
game_folders = list(set(game_folders))

def get_game_sim(gname):
    lst = [cosine_sim(gname, fld) for fld in game_folders]
    return lst

print('%s good games, %s rom folders' % (len(games_names), len(game_folders)))

525 good games, 4863 rom folders


In [5]:
folders_full = []
folders = []

PP = iolib.PrintProgress(games_names)
for g in games_names:
    L = get_game_sim(g)
    folders_full.append(game_folders_full[L.index(max(L))])
    folders.append(game_folders[L.index(max(L))])
    PP.increment()
    #iolb.folder_copy
  

KeyboardInterrupt: 

In [55]:
df_good = pd.DataFrame(list(zip(games_names,games_yr, games_scores,games_votes,games_val, folders_full, folders)), columns=['name', 'yr', 'score','vote','val','folders_full','folders'])
#xlwings.view(df_good)

In [6]:
folders_full

['C:\\temp\\c64\\Games\\C\\Crazy Cars',
 'C:\\temp\\c64\\Games\\A\\Aspar GP Master - Grand Prix Master',
 'C:\\temp\\c64\\Games\\H\\Hacker 2',
 'C:\\temp\\c64\\Games\\I\\I Play 3D Soccer',
 'C:\\temp\\c64\\Games\\H\\Herby',
 'C:\\temp\\c64\\Games\\A\\Ace of Aces',
 'C:\\temp\\c64\\Games\\F\\Feasibility Experiment',
 'C:\\temp\\c64\\Games\\B\\Black It',
 'C:\\temp\\c64\\Games\\C\\Car Wars II',
 'C:\\temp\\c64\\Games\\A\\Augie Doggie And Doggie Daddy',
 'C:\\temp\\c64\\Games\\G\\Grand Prix Simulator 2',
 'C:\\temp\\c64\\Games\\H\\He-man and the Masters of the Universe',
 'C:\\temp\\c64\\Games\\A\\Arkanoid 3',
 'C:\\temp\\c64\\Games\\A\\Arkanoid 3',
 'C:\\temp\\c64\\Games\\E\\Elektrix',
 'C:\\temp\\c64\\Games\\B\\Bomb Jack 2',
 'C:\\temp\\c64\\Games\\A\\Army Days',
 'C:\\temp\\c64\\Games\\C\\Chuckie Egg',
 'C:\\temp\\c64\\Games\\F\\Flintstones, The - Fred Feuerstein',
 'C:\\temp\\c64\\Games\\F\\Flintstones, The - Yabba Dabba Doo!',
 "C:\\temp\\c64\\Games\\D\\David's Midnight Magic",
 'C:\